In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.onnx

In [3]:
class ModularMLP(nn.Module):
    def __init__(self, config):
        super(ModularMLP, self).__init__()
        self.input_size = config['input_size']
        self.layer_size = config['layer_size']
        self.num_hidden_layers = config['num_hidden_size']
        self.output_size = config['output_size']

        layers = []
        layers.append(nn.Linear(self.input_size, self.layer_size))
        layers.append(nn.ReLU())

        for _ in range(self.num_hidden_layers - 1):  # -1 because we want to add the final layer separately
            layers.append(nn.Linear(self.layer_size, self.layer_size))
            layers.append(nn.ReLU())

        layers.append(nn.Linear(self.layer_size, self.output_size))

        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

    def initialize_random_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight)
                nn.init.normal_(m.bias)

# Export the model to ONNX
def export_to_onnx(model, dummy_input, filename):
    torch.onnx.export(model, dummy_input, filename)

In [4]:
config = {
    'input_size': 200,
    'layer_size': 100,
    'num_hidden_size': 10,
    'output_size': 10
}

model = ModularMLP(config)
model.initialize_random_weights()  # Initialize with random weights

# Export the model
dummy_input = torch.randn(1, config['input_size'])  # Single input tensor of shape [1, 200]
export_to_onnx(model, dummy_input, f"mlp_{config['num_hidden_size']}.onnx")

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



## Generate Scarb project

In [7]:
import subprocess
import os


def add_dependencies(path, root=True):
    filepath = f"{path}/Scarb.toml"
    with open(filepath, 'r') as file:
        lines = file.readlines()

    # Search for the [dependencies] section
    index = -1
    for i, line in enumerate(lines):
        if line.strip() == "[dependencies]":
            index = i
            break

    # Add the dependencies after the [dependencies] section
    if index != -1:
        dependencies = [
            'orion = { git = "https://github.com/gizatechxyz/orion.git", rev = "v0.1.1" }\n',]

        if root:
            dependencies.append(
                'model = { path = "model" }\n inputs = { path = "inputs" }\n inference = { path = "inference" }\n')
        for dep in dependencies:
            index += 1
            lines.insert(index, dep)

        with open(filepath, 'w') as file:
            file.writelines(lines)
    else:
        print(f"Could not find [dependencies] section in {filepath}")


def create_scarb_project(name):
    root = f"scarb new {name}"
    subprocess.run(root, shell=True, check=True)
    commands = f"cd {name} && scarb new model && scarb new inputs && scarb new inference"
    subprocess.run(commands, shell=True, check=True)
    add_dependencies(name)
    add_dependencies(f"{name}/model", False)
    add_dependencies(f"{name}/inputs", False)
    add_dependencies(f"{name}/inference", False)
    os.makedirs(f"{name}/model/src/weights", exist_ok=True)
    os.makedirs(f"{name}/model/src/biases", exist_ok=True)


def write_tensor_to_cairo(tensor, tensor_name, directory):
    with open(os.path.join(directory, f"{tensor_name}.cairo"), "w") as f:
        f.write(
            "use array::ArrayTrait;\n" +
            "use orion::operators::tensor::{TensorTrait, Tensor, FP16x16Tensor};\n" +
            "use orion::numbers::FP16x16;\n\n" +
            "\nfn {0}() -> Tensor<FP16x16> ".format(tensor_name) + "{\n" +
            "    let mut shape = ArrayTrait::<usize>::new();\n"
        )
        for dim in tensor.shape:
            f.write("    shape.append({0});\n".format(dim))
        f.write(
            "    let mut data = ArrayTrait::<FP16x16>::new();\n"
        )
        for val in tensor.flatten():
            val_fp = int(val.item() * 2 ** 16)
            f.write("    data.append(FP16x16 {{ mag: {0}, sign: {1} }});\n".format(
                abs(val_fp), str(val_fp < 0).lower()))
        f.write(
            "    TensorTrait::new(shape.span(), data.span())\n" +
            "}\n"
        )

def generate_mod_file(directory):
    with open(f"{directory}.cairo", "w") as f:
        for item_name in os.listdir(directory):
            # Exclude if it's a directory
            if not os.path.isdir(f"{directory}/{item_name}"):
                module_name = item_name.split('.')[0]  # removing file extension
                f.write(f"mod {module_name};\n")


def write_tensors(model, dummy_input, project_name):
    input_tensor = dummy_input[0].flatten()
    write_tensor_to_cairo(input_tensor, "input", f"{project_name}/inputs/src")

    for idx, m in enumerate(model.modules()):
        if isinstance(m, nn.Linear):
            layer_name = f"layer_{idx}"
            weight_name = f"{layer_name}_weights"
            bias_name = f"{layer_name}_bias"

            weight_dir = f"{project_name}/model/src/weights"
            bias_dir = f"{project_name}/model/src/biases"
            
            # Writing weights and biases
            write_tensor_to_cairo(m.weight, weight_name, weight_dir)
            write_tensor_to_cairo(m.bias, bias_name, bias_dir)

            # Generating mod file
            generate_mod_file(weight_dir)
            generate_mod_file(bias_dir)


def write_in_lib(path, content):
     with open(f"{path}.cairo", "w") as f:
         f.write(content)
         
def write_inference(project_name):
    with open(f"{project_name}/inference/src/lib.cairo", "w") as f:
            f.write()
        for _ in range(config['num_hidden_size'] - 1):
            f.write()

# Usage:
project_name = "your_project_name"
create_scarb_project(project_name)
write_tensors(model, dummy_input, project_name)
write_in_lib(f"{project_name}/model/src/lib" ,"mod weights;\nmod biases;") #model
write_in_lib(f"{project_name}/inputs/src/lib", "mod input;") #input

Created `your_project_name` package.
Created `model` package.
Created `inputs` package.
Created `inference` package.
